In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import sys

In [2]:
#this function is used to update data that didn't populate in initial data call above
def update_data(symbol):
    data = yf.download(
            tickers = symbol,
            period = '200d',
            group_by = 'ticker',
            auto_adjust = True,
            threads = True,
            proxy = None)
    return data

In [3]:
stocks = pd.read_csv('cleaned_main_data.csv')
stocks.head()

,Unnamed: 0,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date
0,0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1.593389e+09,1.414973e+09,1398:1000,2020-11-23,2020-07-31 00:00:00
1,1,AA,Alcoa Corp,NYSE,Aluminum,Basic Materials,NaN,NaN,1.477958e+09,1000:801,2020-10-14,2020-06-30 00:00:00
2,2,AAN,"Aaron's, Inc",NYSE,Rental & Leasing Services,Industrials,0.16,1.600214e+09,1.271376e+09,3:2,2020-11-02,2020-06-30 00:00:00
3,3,AAP,"Advance Auto Parts, Inc W/I",NYSE,Specialty Retail,Consumer Cyclical,1.00,1.600301e+09,1.127693e+09,3:2,2020-11-10,2020-07-11 00:00:00
4,4,AAT,"American Assets Trust, Inc",NYSE,REIT—Diversified,Real Estate,1.00,1.599610e+09,NaN,NaN,2020-10-27,2020-06-30 00:00:00


In [4]:
stocks = stocks.drop(stocks.columns[0], axis=1)

In [5]:
# symbols = stocks['Symbol'][:5].tolist()
symbols = stocks['Symbol'].tolist()
type(symbols)

list

In [6]:
len(symbols)

3940

In [7]:
data = yf.download(
tickers = symbols,
period = '200d',
group_by = 'ticker',
auto_adjust = True,
threads = True,
proxy = None)

[*********************100%***********************]  3934 of 3934 completed

5 Failed downloads:
- GLAD: No data found, symbol may be delisted
- THR: No data found, symbol may be delisted
- GABC: No data found, symbol may be delisted
- GBIO: No data found, symbol may be delisted
- GFNCP: No data found, symbol may be delisted


In [8]:
data.head(2)

AVNW                            BLI                        ...  \
             Open  High    Low Close Volume Open High Low Close Volume  ...   
Date                                                                    ...   
2019-11-15  14.30  14.3  13.96  14.1  53100  NaN  NaN NaN   NaN    NaN  ...   
2019-11-18  14.19  14.2  13.96  14.0  10600  NaN  NaN NaN   NaN    NaN  ...   

                 TESS                                                 MDP  \
                 Open       High        Low      Close  Volume       Open   
Date                                                                        
2019-11-15  11.386363  11.645597  11.167011  11.486069   84300  34.140283   
2019-11-18  11.476097  12.832091  11.296627  12.154094  144500  34.457905   

                                                     
                 High        Low      Close  Volume  
Date                                                 
2019-11-15  34.669663  33.687901  34.467537  354000  
2019-11-18  34.506030  33.466520  33.793774  574300  

[2 rows x 19677 columns]

In [9]:
stocks.shape

(3940, 11)

In [10]:
new_stocks = stocks[~(stocks.Symbol.isin(['GABC', 'FWONA', 'GFED', 'NATI']))]

In [11]:
new_stocks.shape

(3936, 11)

In [12]:
stocks.head(2)

,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date
0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1.593389e+09,1.414973e+09,1398:1000,2020-11-23,2020-07-31 00:00:00
1,AA,Alcoa Corp,NYSE,Aluminum,Basic Materials,NaN,NaN,1.477958e+09,1000:801,2020-10-14,2020-06-30 00:00:00


In [13]:
earnings_date_index = stocks.columns.tolist().index('previous_earnings_date')
symbol_index = stocks.columns.tolist().index('Symbol')

In [14]:
indicators = ['BBP', 'Exponential Moving Average']
for indicator in indicators:
    stocks.loc[:, indicator] = np.nan
stocks.head(1)

,Symbol,Name,Exchange,industry,sector,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,upcoming_earnings_date,previous_earnings_date,BBP,Exponential Moving Average
0,A,"Agilent Technologies, Inc",NYSE,Diagnostics & Research,Healthcare,0.72,1.593389e+09,1.414973e+09,1398:1000,2020-11-23,2020-07-31 00:00:00,NaN,NaN


In [15]:
#calculate exponential moving average
def EMA(data, date, days=5, window=20, smoothing=2):
    data = data / data.iloc[0]
    data['SMA'] = data['Close'].rolling(window=window).mean()
    data['EMA'] = data['SMA']
    
    ema_index = data.columns.tolist().index('EMA')
    value_index = data.columns.tolist().index('Close')
    
    i=0
    for row in data.itertuples():
        
        if i == 0:
            previous_row = row
        else:
#             try:
#                 value_today = row[value_index+1]
#                 ema = (value_today *(smoothing/(1.0+days)) 
#                        + previous_row[ema_index+1] *(1.0-(smoothing/(1.0+days))))
#                 data.at[i,'EMA'] = ema
                
#             except:
#                 e = sys.exc_info()[0]
            value_today = row[value_index+1]
            ema = (value_today *(smoothing/(1.0+days)) 
                   + previous_row[ema_index+1] *(1.0-(smoothing/(1.0+days))))
            data.at[row[0],'EMA'] = ema
            previous_row = row
            
        i += 1
    print(data)
    idx = data.index.get_loc(date)
    ema = data['EMA'].iloc[idx-days]
#     print(bbp)
    return ema


In [16]:
#calculate Bollinger Bands
def BBP(data, date, days=5, window=20):

    data['SMA'] = data['Close'].rolling(window=window).mean()
    data['SD'] = data['Close'].rolling(window=window).std()
    data['BB_above'] = data['SMA'] + 2*data['SD']
    data['BB_below'] = data['SMA'] - 2*data['SD']
    data['BBP'] = np.divide(data['Close'] - data['SMA'], 2*data['SD'])

    #get the index of the earnings date
    idx = data.index.get_loc(date)
    
    #
#     print(data[idx-days:idx])
    bbp = data['BBP'].iloc[idx-days]
#     print(bbp)
    return bbp
    
#     print (data['BBP']index)
#     return data['BBP'].loc[date]
#     data[symbol]['Open'].loc[earnings_date]

In [17]:
#calculate momentum
def mom(data, date, days=5, window=20):
    pass
#     data = data / data.iloc[0]
#     data['Close'] = np.divide(data.iloc[window:, 'Close'], data.iloc[::data.shape[0]-window, 'Close'])-1
#     print(data)
    
#     pass

In [18]:
index=0
days=5

for row in stocks.itertuples():
    earnings_date=(row[earnings_date_index+1])
    symbol = row[symbol_index+1]
#     print (data[symbol]['Open'].loc[a])
    
    # some data wasn't populated.  Check for this and populate it if so
    j=0
    while np.isnan(data[symbol]['Open'].loc[earnings_date]):
        data[symbol] = update_data(symbol)
        i += 1
        if j > 5:
            print('Data for ', symbol, ' will not update!')
    
    #get BBP.  Using try because data may not have updated above!!!
    stocks.at[index, 'Exponential Moving Average'] == EMA(data[symbol].copy(), earnings_date, days)

#     for indicator in indicators:
#         try:
#             if indicator == 'BBP':
#                 stocks.at[index,indicator] = BBP(data[symbol].copy(), earnings_date, days)
#             elif indicator == 'Momentum':
#                 stocks.at[index,indicator] == mom(data[symbol].copy(), earnings_date, days)
#             else:
#                 stocks.at[index,indicator] == EMA(data[symbol].copy(), earnings_date, days)
#         except:
#             print('Could not create ', indicator, ' for ', symbol)

    
#     print(initial_date)
#     print (symbol)
#     print (row)
    index +=1
#     break

                Open      High       Low     Close    Volume       SMA  \
Date                                                                     
2019-11-15  1.000000  1.000000  1.000000  1.000000  1.000000       NaN   
2019-11-18  1.015729  1.002426  1.011617  0.995911  0.965185       NaN   
2019-11-19  1.016118  1.015068  1.016055  1.013161  1.002588       NaN   
2019-11-20  1.028987  1.013408  1.020885  1.003322  1.428958       NaN   
2019-11-21  1.026518  1.008428  1.009137  1.000511  1.259821       NaN   
...              ...       ...       ...       ...       ...       ...   
2020-08-26  1.284867  1.282152  1.288526  1.277826  0.630616  1.256786   
2020-08-27  1.298085  1.281252  1.282086  1.273837  0.727417  1.258773   
2020-08-28  1.295598  1.284723  1.294965  1.285158  0.509292  1.261070   
2020-08-31  1.307377  1.298994  1.310866  1.291848  0.633322  1.261925   
2020-09-01  1.320725  1.302208  1.317437  1.300853  0.618519  1.264234   

                 EMA  
Date          

KeyError: '2020-07-11 00:00:00'

In [ ]:
stocks.head(5)